# Praktikum 3

In [ ]:
# Install hnswlib terlebih dahulu.
!pip install hnswlib


In [ ]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")


=== Exact NN ===
Indices: [[820 724  43 896 631]]
Distances: [[0.01405381 0.02617526 0.02949847 0.0450366  0.04665572]]
Waktu: 0.0853116512298584 detik

=== HNSW ===
Indices: [[820 724  43 896 631]]
Distances: [[0.00019751 0.00068514 0.00087016 0.0020283  0.00217676]]
Waktu: 0.0004096031188964844 detik


Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1jt data, 2D vs 5D data. catat hasilnya pada tabel yang anda buat sendiri seperti pada praktikum 1.

| Jumlah Data | Dimensi | Metode                     | Indices                             | Distances                                                                 | Waktu (detik) |
| ----------- | ------- | -------------------------- | ----------------------------------- | ------------------------------------------------------------------------- | ------------- |
| 1,000       | 2D      | **Exact NN (Flat)**        | [[298 433 523 663 642]]             | [[0.00998154 0.01404382 0.01586299 0.03434097 0.0408712]]                 | 0.0023        |
| 1,000       | 2D      | **HNSW (Approx)**          | [[298 433 523 663 642]]             | [[9.9631157e-05 1.9722892e-04 2.5163437e-04 1.1793022e-03 1.6704553e-03]] | 0.00023       |
| 1,000,000   | 5D      | **Exact NN (Brute Force)** | [[601 947668 171282 128993 489096]] | [[0.03225626 0.06055474 0.06217775 0.06631041 0.06739441]]                | 0.0389        |
| 1,000,000   | 5D      | **HNSW (Approx)**          | [[601 947668 171282 128993 489096]] | [[0.00104047 0.00366688 0.00386607 0.00439707 0.00454201]]                | 0.0003        |


In [ ]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ==========================================================
# 1. Generate Data 1 Juta Sampel Berdimensi 5 (5D)
# ==========================================================
num_elements = 1_000_000   # jumlah data
dim = 5                    # jumlah dimensi
data = np.random.random((num_elements, dim)).astype(np.float32)

# Titik query tunggal (bisa dari data atau titik acak)
query = np.random.random((1, dim)).astype(np.float32)
k = 5  # cari 5 tetangga terdekat

# ==========================================================
# 2. Exact Nearest Neighbor (Brute Force)
# ==========================================================
# Gunakan sklearn NearestNeighbors dengan algoritma brute-force (akurasi 100%)
print("=== Exact NN (Brute Force) ===")
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances_exact, indices_exact = nn.kneighbors(query)
end = time.time()
exact_time = end - start

print("Indices:", indices_exact)
print("Distances:", distances_exact)
print("Waktu Exact NN :", round(exact_time, 4), "detik")

# ==========================================================
# 3. Approximate NN dengan HNSW
# ==========================================================
# Inisialisasi index HNSW (Hierarchical Navigable Small World Graph)
print("\n=== Approx NN (HNSW) ===")
p = hnswlib.Index(space='l2', dim=dim)  # space 'l2' = Euclidean distance

# Inisialisasi struktur index
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data ke index (proses build index)
print("Membangun index HNSW...")
p.add_items(data)

# Set ef parameter untuk pencarian (tradeoff antara akurasi & kecepatan)
p.set_ef(100)

# Lakukan query ANN
start = time.time()
labels_ann, distances_ann = p.knn_query(query, k=k)
end = time.time()
ann_time = end - start

print("Indices:", labels_ann)
print("Distances:", distances_ann)
print("Waktu Approx NN (HNSW):", round(ann_time, 4), "detik")

# ==========================================================
# 4. Ringkasan Hasil Perbandingan
# ==========================================================
print("\n=== RINGKASAN PERBANDINGAN ===")
print(f"Jumlah data\t\t: {num_elements}")
print(f"Dimensi data\t\t: {dim}D")
print(f"Waktu Exact NN\t\t: {exact_time:.4f} detik")
print(f"Waktu HNSW ANN\t\t: {ann_time:.4f} detik")



=== Exact NN (Brute Force) ===
Indices: [[   601 947668 171282 128993 489096]]
Distances: [[0.03225626 0.06055474 0.06217775 0.06631041 0.06739441]]
Waktu Exact NN : 0.0389 detik

=== Approx NN (HNSW) ===
Membangun index HNSW...
Indices: [[   601 947668 171282 128993 489096]]
Distances: [[0.00104047 0.00366688 0.00386607 0.00439707 0.00454201]]
Waktu Approx NN (HNSW): 0.0003 detik

=== RINGKASAN PERBANDINGAN ===
Jumlah data		: 1000000
Dimensi data		: 5D
Waktu Exact NN		: 0.0389 detik
Waktu HNSW ANN		: 0.0003 detik
